In [2]:
import math
import numpy as np
from numba import cuda, jit

@cuda.jit
def mandel_numba_cuda(an_array,r1,r2):
    x, y = cuda.grid(2)
    if x < an_array.shape[0] and y < an_array.shape[1]:
        z = complex( r1[x], r2[y])
        c = z
        for n in range(80):
            if abs(z) > 2:
                an_array[x,y]=n
            z = z*z + c
        an_array[x,y]=n
    return

threadsperblock = (32, 32)

@jit
def mandelperf_numba_cuda():
    r1 = np.linspace(-2.0, 0.5, 260, dtype=float)
    r2 = np.linspace(-1.0, 1.0, 210, dtype=float)
    an_array=np.zeros((r1.shape[0],r2.shape[0]), dtype=int)
    blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
    blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    mandel_numba_cuda[blockspergrid, threadsperblock](an_array,r1,r2)
    return an_array

%timeit mandelperf_numba_cuda()

The slowest run took 496.59 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 3.97 ms per loop
